In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/rolandmueller/titanic/main/titanic3.csv")
X = data.drop(["sex","cabin","boat",'home.dest',"body","embarked","ticket"], axis = 1)
y = data["sex"]
X["age"]=X["age"].fillna(X["age"].mean())
# X=X.dropna(axis=0,how="all")
X["fare"]=X["fare"].fillna(X["fare"].mean())

In [3]:
y = y.map({'female': 0, 'male': 1})

In [4]:
X

,pclass,survived,name,age,sibsp,parch,fare
0,1,1,"Allen, Miss. Elisabeth Walton",29.000000,0,0,211.3375
1,1,1,"Allison, Master. Hudson Trevor",0.916700,1,2,151.5500
2,1,0,"Allison, Miss. Helen Loraine",2.000000,1,2,151.5500
3,1,0,"Allison, Mr. Hudson Joshua Creighton",30.000000,1,2,151.5500
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",25.000000,1,2,151.5500
...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",14.500000,1,0,14.4542
1305,3,0,"Zabour, Miss. Thamine",29.881135,1,0,14.4542
1306,3,0,"Zakarian, Mr. Mapriededer",26.500000,0,0,7.2250
1307,3,0,"Zakarian, Mr. Ortin",27.000000,0,0,7.2250


In [5]:
import re
def clean_data(text):
    text = re.sub(r'Mrs\.', ' ', text)
    text = re.sub(r'Mr\.', ' ', text)
    text = re.sub(r'Miss\.', ' ', text)
    
    text = text.lower()
    return text


In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   int64  
 1   survived  1309 non-null   int64  
 2   name      1309 non-null   object 
 3   age       1309 non-null   float64
 4   sibsp     1309 non-null   int64  
 5   parch     1309 non-null   int64  
 6   fare      1309 non-null   float64
dtypes: float64(2), int64(4), object(1)
memory usage: 71.7+ KB


In [7]:
X['name'] = X['name'].apply(clean_data)

In [15]:
from sklearn.model_selection import train_test_split

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [16]:
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(X['name'])

In [17]:
x = pd.DataFrame(x.toarray())

In [18]:
X_ = pd.concat([X,x], axis=1)

In [19]:
X_= X_.drop("name", axis = 1) 

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2, random_state=42)

In [21]:
from sklearn.linear_model import LogisticRegression
from  sklearn.metrics import *

In [22]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1047 entries, 772 to 1126
Columns: 1935 entries, pclass to 1928
dtypes: float64(1931), int64(4)
memory usage: 15.5 MB


In [23]:

clf = LogisticRegression()
clf.fit(X_train.values, y_train)
print(classification_report(clf.predict(X_test.values), y_test))
print(accuracy_score(y_test, clf.predict(X_test.values)))

              precision    recall  f1-score   support

           0       0.80      0.71      0.75       108
           1       0.81      0.88      0.84       154

    accuracy                           0.81       262
   macro avg       0.81      0.79      0.80       262
weighted avg       0.81      0.81      0.81       262

0.8091603053435115


/opt/homebrew/Caskroom/miniforge/base/envs/innopolis_ml/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
import catboost as cb

In [25]:
model = cb.CatBoostClassifier()
model.fit(X_train, y_train)

Learning rate set to 0.010506
0:	learn: 0.6883926	total: 60.8ms	remaining: 1m
1:	learn: 0.6826688	total: 62.2ms	remaining: 31s
2:	learn: 0.6782204	total: 63.6ms	remaining: 21.1s
3:	learn: 0.6729539	total: 64.9ms	remaining: 16.2s
4:	learn: 0.6674032	total: 66.3ms	remaining: 13.2s
5:	learn: 0.6632418	total: 67.8ms	remaining: 11.2s
6:	learn: 0.6583842	total: 69ms	remaining: 9.79s
7:	learn: 0.6544404	total: 70.2ms	remaining: 8.7s
8:	learn: 0.6502974	total: 71.8ms	remaining: 7.9s
9:	learn: 0.6462935	total: 73ms	remaining: 7.23s
10:	learn: 0.6398221	total: 74.5ms	remaining: 6.7s
11:	learn: 0.6340939	total: 76.1ms	remaining: 6.27s
12:	learn: 0.6296289	total: 77.5ms	remaining: 5.89s
13:	learn: 0.6259859	total: 79ms	remaining: 5.56s
14:	learn: 0.6223842	total: 80.3ms	remaining: 5.27s
15:	learn: 0.6185257	total: 81.7ms	remaining: 5.02s
16:	learn: 0.6144162	total: 83.1ms	remaining: 4.8s
17:	learn: 0.6106270	total: 84.5ms	remaining: 4.61s
18:	learn: 0.6068631	total: 85.7ms	remaining: 4.42s
19:	lea

In [26]:
print(accuracy_score(y_test, model.predict(X_test.values)))

0.8396946564885496


In [27]:
import optuna
import numpy as np

In [29]:
def objective(trial):
    
    
    X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.3, random_state=42)

    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    cat_cls = cb.CatBoostClassifier(**param)

    cat_cls.fit(X_train, y_train, eval_set=[(X_test, y_test)],verbose=0, early_stopping_rounds=100)

    preds = cat_cls.predict(X_test)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_test, pred_labels)
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-03-02 09:15:42,729] A new study created in memory with name: no-name-4af652e2-fcb3-4150-a1f2-29feafaa8e66
[I 2024-03-02 09:16:30,039] Trial 0 finished with value: 0.8447837150127226 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.01301915146814625, 'depth': 11, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 7.839636841096786}. Best is trial 0 with value: 0.8447837150127226.
[I 2024-03-02 09:16:30,743] Trial 1 finished with value: 0.8473282442748091 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.08317317594284705, 'depth': 3, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.9929211186359047}. Best is trial 1 with value: 0.8473282442748091.
[I 2024-03-02 09:16:43,768] Trial 2 finished with value: 0.8676844783715013 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.08547486174300041, 'depth': 9, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 2 with

Number of finished trials: 50
Best trial:
  Value: 0.8829516539440203
  Params: 
    objective: Logloss
    colsample_bylevel: 0.018113144107355195
    depth: 6
    boosting_type: Ordered
    bootstrap_type: MVS


In [33]:

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Определяем классификаторы
logistic = LogisticRegression()
svm = SVC()

# Определяем ансамбль
eclf = VotingClassifier(estimators=[('lr', logistic), ('svm', svm)], voting='hard')

# Обучаем ансамбль на данных
eclf.fit(X_train.values, y_train)

# Оцениваем точность на тестовых данных
y_pred = eclf.predict(X_test.values)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

/opt/homebrew/Caskroom/miniforge/base/envs/innopolis_ml/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8129770992366412
